# Проект 3 о вкусной и здоровой пище

  В данном проекте представлен датасет содержащий информацию о ресторанах, которые есть в базе сайта Trip Advisor.
В рамках данного учебного проекта необходимо проанализировать данные, заполнить пропуски, создать новые признаки для 
предсказания рейтинга ресторана.
 Решение в данном ноутбуке отличается от того что я выложил на кагл. https://www.kaggle.com/nikolaylukichev/project-3-tasty-and-healthy
 В данном ноутбуке получилось добиться значения метрики примерно 0,08, но создать файл submission не удалось, поэтому на каггл ушло решение с меньшим количеством признаков, но и худшим значением метрики (0,217)
 В данном решении так же не представлен анализ в виде графиков и таблиц корреляции, в чем сразу каюсь, но минимум выполнен - результат получен, надеюсь получить проходной балл))

In [1]:
import pandas as pd
import re
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

In [2]:
RANDOM_SEED = 41

In [3]:
data_fs = pd.read_csv('main_task.csv')
data_fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


# Предобработка

In [4]:
# Функция для замены уровня цен ресторана числовым значением. Условно положим, что $ = 1, $$ - $$$ = 2.5, $$$$ = 4
def replace_range (s):
    if s == '$':
        return 1
    elif s == '$$ - $$$':
        return 2.5
    elif s == '$$$$':
        return 4
    
# Функция для вычленения данных о дате отзыва из колонки Reviews
def my_date (row):
    pattern = re.compile('\d\d\/\d\d\/\d\d\d\d')
    return pattern.findall(row)

# Функция для перевода в формат datetime
def str_to_date (list_d):
    real_date = []
    for i in list_d:
        one_date = datetime.strptime(i, '%m/%d/%Y')
        real_date.append(one_date)
    return real_date

# Функция для поиска положительного прилагательного из описания Reviews
def good_adj (row):
    c = 1
    if 'good' in row or 'great' in row or 'nice' in row or 'better' in row or 'best' in row \
    or 'delicious' in row or 'amazing' in row or 'excellent' in row or 'super' in row or 'wonderful' in row or 'yummy' in row:
        c+=1
    return c

# Функция для поиска отрицательного прилагательного из описания Reviews
def bad_adj (row):
    c = 1
    if 'bad' in row or 'worse' in row or 'worst' in row or 'terible' in row or 'awful' in row or 'poor' in row \
    or 'unacceptable' in row or 'sad' in row or 'junky' in row or 'shameful' in row:
        c-=1 
    return c 

# Функция для поиска вегетарианской кухни
def veg (row):
    c = 0
    for i in row:
        if 'Vegetarian Friendly' in i:
            c+=1
        return c    

In [5]:
# Создадим новую колонку с количественным признаком определяющим дороговизну ресторана
data_fs['quant_price'] = data_fs['Price Range'].apply(replace_range)

# Посчитаем среднее значение и добавим его вместо пропусков
data_fs['quant_price'].fillna(round(data_fs['quant_price'].mean(), 2), inplace = True)
data_fs.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50


In [6]:
# Создадим временный датасет, удалим из него строки с пропусками в колонке 'Cuisine Style' 
for_del = data_fs.copy()
for_del.dropna(subset=['Cuisine Style'], inplace=True)
for_del['Cuisine Style'] = for_del['Cuisine Style'].str.split(', ')

# Создадим новую колонку с количеством типов кухни в каждом ресторане
for_del['quant_cuisine'] = for_del['Cuisine Style'].apply(lambda x: len(x))
# Узнаем среднее значение типов кухни, его потом будем подставлять в пропуски
mean_cuisine = round(for_del['quant_cuisine'].mean())
mean_cuisine

3

In [7]:
# Мы выяснили что среднее значение для кухонь равно трем, поэтому заполним пропуски строчкой в которой 3 объекта разделены ', '
data_fs['Cuisine Style'].fillna('[Xy, Ya, Se]', inplace = True)

# Уберем квадратные скобки в начале и в конце строки (пригодится позже для создания Дамми переменных)
data_fs['Cuisine Style'] = data_fs['Cuisine Style'].apply(lambda x: x[1:-1])

# Превратим строку в список в качестве разделителя укажем запятую с пробелом 
data_fs['Cuisine Style'] = data_fs['Cuisine Style'].str.split(', ')

# Создадим новую колонку с количеством типов кухни в каждом ресторане
data_fs['quant_cuisine'] = data_fs['Cuisine Style'].apply(lambda x: len(x))
data_fs.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,quant_cuisine
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,3
1,id_1535,Stockholm,"[Xy, Ya, Se]",1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,3
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00,7
3,id_3456,Berlin,"[Xy, Ya, Se]",3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22,3
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50,3


In [8]:
# Вычленяем даты из строки при помощи функции работающей с регулярными выражениями
data_fs['review_date'] = data_fs['Reviews'].apply(my_date)

# Приводим к формату даты
data_fs['review_date'] = data_fs['review_date'].apply(str_to_date)

# Вычленяем из списков значений дат собственно значения и прписываем их в двух новых колонках
data_fs['review_date1'] = data_fs['review_date'].apply(lambda x: x[0] if len(x)==2 else None)
data_fs['review_date2'] = data_fs['review_date'].apply(lambda x: x[1] if len(x)==2 else None)

# Формируем новый полиноминальный признак как разницу в днях между датами написания двух отзывов
data_fs['diff_date'] = abs(data_fs['review_date1'] - data_fs['review_date2'])

# Заполняем пропуски средним значением
data_fs['diff_date'].fillna(data_fs['diff_date'].mean(), inplace=True)

# Переводим из формата Timedelta в int, используя только значения дней (в средних есть еще и часы, минуты, секунды)
data_fs['diff_date'] = data_fs['diff_date'].dt.days.astype('int16')
data_fs.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,quant_cuisine,review_date,review_date1,review_date2,diff_date
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,3,"[2017-12-31 00:00:00, 2017-11-20 00:00:00]",2017-12-31,2017-11-20,41
1,id_1535,Stockholm,"[Xy, Ya, Se]",1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,3,"[2017-07-06 00:00:00, 2016-06-19 00:00:00]",2017-07-06,2016-06-19,382
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00,7,"[2018-01-08 00:00:00, 2018-01-06 00:00:00]",2018-01-08,2018-01-06,2
3,id_3456,Berlin,"[Xy, Ya, Se]",3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22,3,[],NaT,NaT,141
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50,3,"[2017-11-18 00:00:00, 2017-02-19 00:00:00]",2017-11-18,2017-02-19,272


In [9]:
# Приведем все символы в колонке Reviews к нижнему регистру
data_fs['Reviews'] = data_fs['Reviews'].str.lower()

# Создадим две новых колонки в одной, если в описании есть положительные прилагательные, значение будет равно '2' иначе '1'
# в другой, если в описании есть отрицательные прилагательные, значение будет равно '0' иначе '1'
# для этого используем написанные ранее функции
data_fs['good_rest'] = data_fs['Reviews'].apply(good_adj)
data_fs['bad_rest'] = data_fs['Reviews'].apply(bad_adj)

# Сформируем новый признак сложив две созданные колонки. Здесь 2+2=4, 2+1=3 - хороший отзыв, 1+1=2 или 2+0=2 - нейтральный отзыв
# 1+0=1 или 0+0=0 плохой отзыв
data_fs['good_bad_rest'] = data_fs['good_rest'] + data_fs['bad_rest']
data_fs.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,quant_cuisine,review_date,review_date1,review_date2,diff_date,good_rest,bad_rest,good_bad_rest
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,3,"[2017-12-31 00:00:00, 2017-11-20 00:00:00]",2017-12-31,2017-11-20,41,2,1,3
1,id_1535,Stockholm,"[Xy, Ya, Se]",1537.0,4.0,NaN,10.0,"[['unique cuisine', 'delicious nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,3,"[2017-07-06 00:00:00, 2016-06-19 00:00:00]",2017-07-06,2016-06-19,382,2,1,3
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['catch up with friends', 'not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00,7,"[2018-01-08 00:00:00, 2018-01-06 00:00:00]",2018-01-08,2018-01-06,2,1,1,2
3,id_3456,Berlin,"[Xy, Ya, Se]",3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22,3,[],NaT,NaT,141,1,1,2
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['best place to try a bavarian food', 'nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50,3,"[2017-11-18 00:00:00, 2017-02-19 00:00:00]",2017-11-18,2017-02-19,272,2,1,3


In [10]:
# Создадим признак показывающий наличие или отсутствие вегетаринской кухни
data_fs['veg_cuisine'] = data_fs['Cuisine Style'].apply(veg)
data_fs.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,quant_cuisine,review_date,review_date1,review_date2,diff_date,good_rest,bad_rest,good_bad_rest,veg_cuisine
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,3,"[2017-12-31 00:00:00, 2017-11-20 00:00:00]",2017-12-31,2017-11-20,41,2,1,3,0
1,id_1535,Stockholm,"[Xy, Ya, Se]",1537.0,4.0,NaN,10.0,"[['unique cuisine', 'delicious nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,3,"[2017-07-06 00:00:00, 2016-06-19 00:00:00]",2017-07-06,2016-06-19,382,2,1,3,0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['catch up with friends', 'not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00,7,"[2018-01-08 00:00:00, 2018-01-06 00:00:00]",2018-01-08,2018-01-06,2,1,1,2,0
3,id_3456,Berlin,"[Xy, Ya, Se]",3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22,3,[],NaT,NaT,141,1,1,2,0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['best place to try a bavarian food', 'nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50,3,"[2017-11-18 00:00:00, 2017-02-19 00:00:00]",2017-11-18,2017-02-19,272,2,1,3,0


In [11]:
# Заполним пропуски колонки Number of Reviews ее средним значением
data_fs['Number of Reviews'].fillna(round(data_fs['Number of Reviews'].mean()), inplace = True)

In [12]:
# Добавляем дамми переменные для городов
data_fs = pd.get_dummies(data_fs, columns=[ 'City',], dummy_na=True)
data_fs.head()

,Restaurant_id,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_5569,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,"[Xy, Ya, Se]",1537.0,4.0,NaN,10.0,"[['unique cuisine', 'delicious nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,...,0,0,0,0,0,1,0,0,0,0
2,id_352,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['catch up with friends', 'not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,4.00,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,"[Xy, Ya, Se]",3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.22,...,0,0,0,0,0,0,0,0,0,0
4,id_615,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['best place to try a bavarian food', 'nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.50,...,0,0,0,0,0,0,0,0,0,0


Далее идет часть решения не попавшая на каггл. Я создал дамми переменные из кухонь, но для этого применил метод explode, при этом датасет невероятно распух в высоту, метрика после этого получилась очень хорошей но тестовый файл для модели не стыкуется с файлом submission необходимым для каггла. Возможно это даже читерство)) я удалять эту часть работы не стал, но в зачет она наверное пойти не может. 

In [13]:
data_fsi = data_fs.explode('Cuisine Style')
data_fsi.head()

,Restaurant_id,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,quant_price,...,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich,City_nan
0,id_5569,'European',5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,...,0,0,1,0,0,0,0,0,0,0
0,id_5569,'French',5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,...,0,0,1,0,0,0,0,0,0,0
0,id_5569,'International',5570.0,3.5,$$ - $$$,194.0,"[['good food at your doorstep', 'a good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.50,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,Xy,1537.0,4.0,NaN,10.0,"[['unique cuisine', 'delicious nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,...,0,0,0,0,0,1,0,0,0,0
1,id_1535,Ya,1537.0,4.0,NaN,10.0,"[['unique cuisine', 'delicious nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.22,...,0,0,0,0,0,1,0,0,0,0


In [14]:
data_fsi = pd.get_dummies(data_fsi, columns = ['Cuisine Style',], dummy_na=True)


In [15]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = data_fsi.drop(['Rating','Restaurant_id', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
                 'review_date1', 'review_date2', 'review_date', 'good_rest', 'bad_rest', 
                 ], axis = 1)  
y = data_fsi['Rating'] 

In [16]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [17]:
    # Создаём модель  
regr = RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED)  
      
    # Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

In [18]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.08136234043931831
